In [1]:
def get_predictions(record):
    try:
        s3_client.download_file(os.environ['BUCKET_NAME'], os.environ['S3_EBD_FOLDER']+"embedding_model_v2_develop.h5", '/tmp/embedding_model_v2_develop.h5')
    except:
        return {"message":"not found"}

    model = load_model('/tmp/embedding_model_v2_develop.h5')
    
    text1=pd.DataFrame()
    text1["category"]=record.category
    text2=pd.DataFrame()
    text2["sub_category"]=record.sub_category
    text3=pd.DataFrame()
    text3["city"]=record.city
    text4=pd.DataFrame()
    text4["address"]=record.address
    text5=pd.DataFrame()
    text5["land_title"]=record.land_title

    numeric=record.drop(columns=["category","sub_category","city","address","land_title"])
    numeric_max_scaled = numeric.copy()

    for column in numeric_max_scaled.columns:
        numeric_max_scaled[column] = numeric_max_scaled[column]  / numeric_max_scaled[column].abs().max()
        numeric_max_scaled["price"] = numeric_max_scaled["price"] * 0.8
        numeric_max_scaled["landSize"] = numeric_max_scaled["landSize"] * 0.6
        numeric_max_scaled["bedRooms"] = numeric_max_scaled["bedRooms"] * 0.5
        numeric_max_scaled["washRooms"] = numeric_max_scaled["washRooms"] * 0.4

    numeric=numeric_max_scaled

    text1=text1.values.flatten()
    text2=text2.values.flatten()
    text3=text3.values.flatten()
    text4=text4.values.flatten()
    text5=text5.values.flatten()
    numeric_max_scaled=numeric_max_scaled.values

    text1 = text1 * 1
    text2 = text2 * 0.9
    text3 = text3 * 0.7
    text4 = text4 * 0.3
    text5 = text5 * 0.2

    text_matrix1=tokenization(text1,1)
    text_matrix2=tokenization(text2,2)
    text_matrix3=tokenization(text3,3)
    text_matrix4=tokenization(text4,4)
    text_matrix5=tokenization(text5,5)

    text_matrix_tensor1 = create_input_tensor(text_matrix1)
    text_matrix_tensor2 = create_input_tensor(text_matrix2)
    text_matrix_tensor3 = create_input_tensor(text_matrix3)
    text_matrix_tensor4 = create_input_tensor(text_matrix4)
    text_matrix_tensor5 = create_input_tensor(text_matrix5)

    numeric_max_scaled_tensor = tf.convert_to_tensor(numeric_max_scaled, dtype=tf.float32)
    print(text_matrix_tensor1.shape, text_matrix_tensor2.shape, text_matrix_tensor3.shape, text_matrix_tensor4.shape, text_matrix_tensor5.shape, numeric_max_scaled_tensor.shape)
    prediction=model.predict([ text_matrix_tensor1, text_matrix_tensor2, text_matrix_tensor3, text_matrix_tensor4, text_matrix_tensor5, numeric_max_scaled_tensor])

    return prediction










































# load_dotenv()
# session = boto3.Session(aws_access_key_id=os.environ["aws_access_key_id"], aws_secret_access_key=os.environ["aws_secret_access_key"])
# s3 = session.resource('s3')
# s3_client = session.client('s3')


# def get_predictions(record):
#     try:
#         s3_client.download_file(os.environ['BUCKET_NAME'],
#                                 os.environ['S3_EBD_FOLDER']+"embedding_model_v2_develop.h5",
#                                    '/embedding_model_v2_develop.h5')
#     except:
#         return {"message":"not found"}

#     model = load_model('/embedding_model_v2_develop.h5')
    
    # text1=pd.DataFrame()
    # text1["category"]=record.category
    # text2=pd.DataFrame()
    # text2["sub_category"]=record.sub_category
    # text3=pd.DataFrame()
    # text3["city"]=record.city
    # text4=pd.DataFrame()
    # text4["address"]=record.address
    # text5=pd.DataFrame()
    # text5["land_title"]=record.land_title

    # numeric=record.drop(columns=["category","sub_category","city","address","land_title"])
    # numeric_max_scaled = numeric.copy()

    # for column in numeric_max_scaled.columns:
    #     numeric_max_scaled[column] = numeric_max_scaled[column]  / numeric_max_scaled[column].abs().max()
    #     numeric_max_scaled[column] = numeric_max_scaled[column] * column + 0.3

    # numeric=numeric_max_scaled
    # print(numeric)
    # return numeric

# print("Welcome ",Auth['email'])
conn=initiate_dbaccess()
print("Database connected successfully")

cur = conn.cursor()
cur.execute("""SELECT lambda_count."recommendation" FROM lambda_count WHERE lambda_count."id"=1 """)
row=cur.fetchall()
print(row[0][0])
c1=row[0][0]+1
cur = conn.cursor()
cur.execute("""UPDATE lambda_count SET recommendation = (%s) WHERE lambda_count."id"=1""",(c1,))
conn.commit()

UserID= 503 #Auth['id']
print("User ID: " , UserID)

cur = conn.cursor()  

cur.execute("""SELECT IR."inventoryId",IR."type" FROM inventory_recomendation as IR WHERE
                IR."userId" = (%s) ORDER BY IR."createdAt" """,(UserID,))

distinct = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
dis = pd.DataFrame(distinct, columns = colnames)
df_no_duplicates = dis[~dis.duplicated(keep='last')]
rows = df_no_duplicates.tail(4).values

inv_id=df_no_duplicates["inventoryId"].tail(4)

listing_ids = []
hotlisting_ids = []

df=pd.DataFrame(rows,columns=colnames)
df["type"]=[typ.lower() for typ in df["type"].values]
df1=df.groupby('type')['inventoryId'].apply(list).reset_index(name="inventories")

row1,row2 = [],[]

if "listing" in df1["type"].values:
    listing_ids=df1[df1["type"]=="listing"]['inventories'].values[0]

    cur = conn.cursor()
    cur.execute("""
        SELECT listing_invs.*, LA."title" as land_title FROM land_area as LA JOIN
        (SELECT inv_list.*, pro."city", pro."address" FROM project as pro JOIN
        (SELECT invents.*, list."saleCommission" FROM listing as list JOIN
        (SELECT invent.*, pst."title" as sub_category FROM project_sub_type as pst JOIN
        (SELECT inv."id", inv."projectSubTypeId", inv."projectId", inv."landAreaId",
        inv."price", inv."washRooms", inv."bedRooms", inv."landSize",
        pt."title" as category FROM inventory as inv JOIN
        project_type as pt ON inv."projectTypeId"=pt."id") as invent
        ON pst."id"=invent."projectSubTypeId") as invents
        ON list."inventoryId"=invents."id" WHERE list."status"=(%s)
        AND list."deletedAt" IS NULL) as inv_list
        ON inv_list."projectId"=pro."id") as listing_invs
        ON listing_invs."landAreaId"=LA."id" AND listing_invs."id" IN %s
        """, ('Approved', tuple(listing_ids)))
    row1 = cur.fetchall()

if "hotlisting" in df1["type"].values:
    hotlisting_ids=df1[df1["type"]=="hotlisting"]['inventories'].values[0]
    cur = conn.cursor()
    cur.execute("""
        SELECT listing_invs.*, LA."title" as land_title FROM land_area as LA JOIN
        (SELECT inv_list.*, pro."city", pro."address" FROM project as pro JOIN
        (SELECT invents.*, hotlist."saleCommission" FROM hot_listing as hotlist JOIN
        (SELECT invent.*, pst."title" as sub_category FROM project_sub_type as pst JOIN
        (SELECT inv."id", inv."projectSubTypeId", inv."projectId", inv."landAreaId",
        inv."price", inv."washRooms", inv."bedRooms", inv."landSize",
        pt."title" as category FROM inventory as inv JOIN
        project_type as pt ON inv."projectTypeId"=pt."id") as invent
        ON pst."id"=invent."projectSubTypeId") as invents
        ON hotlist."inventoryId"=invents."id" WHERE hotlist."status"=(%s)
        AND hotlist."deletedAt" IS NULL) as inv_list
        ON inv_list."projectId"=pro."id") as listing_invs
        ON listing_invs."landAreaId"=LA."id" AND listing_invs."id" IN %s
        """, ('Approved', tuple(hotlisting_ids)))
    row2 = cur.fetchall()

print("listing_ids: ", listing_ids)
print("hotlisting_ids: ", hotlisting_ids)



colnames = [desc[0] for desc in cur.description]
row=row1+row2

if row!=[]:
    colnames = [desc[0] for desc in cur.description]
    agency_inventory = pd.DataFrame(row, columns = colnames)
    agency_inventory=agency_inventory.fillna(0)
    if 0 in agency_inventory['category'].values: agency_inventory['category'].replace(0,'',inplace=True)
    if 0 in agency_inventory['sub_category'].values: agency_inventory['sub_category'].replace(0,'',inplace=True)
    if 0 in agency_inventory['city'].values: agency_inventory['city'].replace(0,'',inplace=True)
    if 0 in agency_inventory['address'].values: agency_inventory['address'].replace(0,'',inplace=True)
    if 0 in agency_inventory['land_title'].values: agency_inventory['land_title'].replace(0,'',inplace=True)

    agency_inventory = agency_inventory.drop(columns=['projectSubTypeId','landAreaId'])

else:
    print("none")

final_result = agency_inventory
print(agency_inventory.head())


text1=pd.DataFrame()
text1["category"]=final_result.category
text2=pd.DataFrame()
text2["sub_category"]=final_result.sub_category
text3=pd.DataFrame()
text3["city"]=final_result.city
text4=pd.DataFrame()
text4["address"]=final_result.address
text5=pd.DataFrame()
text5["land_title"]=final_result.land_title

numeric=final_result.drop(columns=["category","sub_category","city","address","land_title"])
numeric_max_scaled = numeric.copy()

for column in numeric_max_scaled.columns:
    numeric_max_scaled[column] = numeric_max_scaled[column]  / numeric_max_scaled[column].abs().max()
    numeric_max_scaled[column] = numeric_max_scaled[column] * 0.3


print(numeric_max_scaled.values)
numeric=numeric_max_scaled

numeric["price"] = numeric["price"] * 0.8
numeric["landSize"] = numeric["landSize"] * 0.6
numeric["bedRooms"] = numeric["bedRooms"] * 0.5
numeric["washRooms"] = numeric["washRooms"] * 0.4


print("="*100)
print(numeric)

NameError: name 'initiate_dbaccess' is not defined